In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
import json
from bs4 import BeautifulSoup
import urllib3

Using TensorFlow backend.


In [2]:
http = urllib3.PoolManager()
response = http.request('GET', 'https://s3.amazonaws.com/csye7374/config.json')
doc = str(BeautifulSoup(response.data))

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [0]:
data = json.loads(doc)

In [4]:
batch_size = data['fit']['Batch_Size']
print(batch_size)
num_classes = data['fit']['Num_Classes']
print(num_classes)
epochs = data['fit']['epochs']
print(epochs)
data_augmentation = data['fit']['Data_Augmentation']
print(data_augmentation)
lr=data['compile']['Learning_Rate']
print(lr)
decay=data['compile']['Decay']
print(decay)

32
10
60
False
0.0001
1e-06


In [5]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

170500096/170498071 [==============================] - 254s 1us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [0]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
model1 = Sequential()
model1.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model1.add(Activation('elu'))
model1.add(Conv2D(32, (3, 3)))
model1.add(Activation('elu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.2))

model1.add(Conv2D(64, (3, 3), padding='same'))
model1.add(Activation('elu'))
model1.add(Conv2D(64, (3, 3)))
model1.add(Activation('elu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.2))

model1.add(Conv2D(128, (3, 3), padding='same'))
model1.add(Activation('elu'))
model1.add(Conv2D(128, (3, 3)))
model1.add(Activation('elu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.2))

model1.add(Flatten())
model1.add(Dense(512))
model1.add(Activation('elu'))
model1.add(Dropout(0.3))
model1.add(Dense(num_classes))
model1.add(Activation('softmax'))

In [12]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
__________

In [0]:
opt = keras.optimizers.rmsprop(lr=lr, decay=decay)

In [0]:
# Let's train the model using RMSprop
model1.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


In [10]:
history = model1.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=data['fit']['Shuffle'])

Train on 50000 samples, validate on 10000 samples
Epoch 1/60
50000/50000 [==============================] - 28s 550us/step - loss: 1.6842 - acc: 0.3866 - val_loss: 1.4253 - val_acc: 0.4849
Epoch 2/60
50000/50000 [==============================] - 24s 478us/step - loss: 1.3568 - acc: 0.5140 - val_loss: 1.2268 - val_acc: 0.5645
Epoch 3/60
50000/50000 [==============================] - 24s 478us/step - loss: 1.2094 - acc: 0.5728 - val_loss: 1.1419 - val_acc: 0.5910
Epoch 4/60
50000/50000 [==============================] - 24s 476us/step - loss: 1.1139 - acc: 0.6066 - val_loss: 1.0372 - val_acc: 0.6363
Epoch 5/60
50000/50000 [==============================] - 24s 478us/step - loss: 1.0437 - acc: 0.6338 - val_loss: 0.9756 - val_acc: 0.6570
Epoch 6/60
50000/50000 [==============================] - 24s 477us/step - loss: 0.9857 - acc: 0.6576 - val_loss: 0.9395 - val_acc: 0.6716
Epoch 7/60
50000/50000 [==============================] - 24s 477us/step - loss: 0.9337 - acc: 0.6742 - val_loss: 0.